# Etapa 2 — Pré-processamento dos Dados  
Dataset: *youtube_views.csv*

Este notebook realiza:
- Tratamento de valores faltantes  
- Tratamento de outliers  
- Encoding de variáveis categóricas  
- Normalização (scaling)  
- Criação opcional de novas features  

## 1. Importação de Bibliotecas

Aqui importamos todas as bibliotecas necessárias para pré-processamento dos dados.

In [13]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)


## 2. Carregamento dos Dados

O arquivo será carregado da pasta `data/datasets/youtube_views.csv`.  
Também exibimos uma prévia para garantir que tudo está correto.


In [14]:
path = "../data/datasets/youtube_views.csv"

df = pd.read_csv(path)
df.head()


,video_id,duration_minutes,title_length,description_length,tags_count,has_thumbnail_custom,video_quality,category,language,has_subtitles,channel_subscribers,channel_age_months,previous_videos_count,avg_upload_frequency_days,comments_count,likes_count,shares_count,playlist_adds,promoted,upload_time,upload_day,total_views
0,VID00197,3,53,589.0,34.0,Sim,720p,Fitness,Português,Não,474,NaN,406,9,2804.0,5269,NaN,624,Não,Tarde,Sexta,2219779
1,VID00987,60,95,NaN,17.0,Não,720p,Culinária,Português,Não,899904,NaN,25,14,9634.0,9448,1679.0,894,Sim,Tarde,Sexta,719904
2,VID01091,47,35,168.0,20.0,Não,1080p,Gaming,Espanhol,Não,761732,59.0,59,1,6943.0,34028,542.0,540,Não,Manhã,Seg-Qui,1420456
3,VID00388,8,12,260.0,4.0,Sim,1080p,Tecnologia,Português,Sim,154261,84.0,93,10,9427.0,42695,3096.0,188,Não,Madrugada,Sábado,329746
4,VID02222,56,94,648.0,48.0,Não,720p,Vlogs,Espanhol,Não,843352,6.0,205,13,2993.0,40761,311.0,957,Não,Manhã,Domingo,984985


## 3. Identificação dos Tipos de Variáveis

Separação das features numéricas e categóricas para aplicar técnicas adequadas em cada grupo.


In [15]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'bool']).columns.tolist()

numeric_cols, categorical_cols


(['duration_minutes',
  'title_length',
  'description_length',
  'tags_count',
  'channel_subscribers',
  'channel_age_months',
  'previous_videos_count',
  'avg_upload_frequency_days',
  'comments_count',
  'likes_count',
  'shares_count',
  'playlist_adds',
  'total_views'],
 ['video_id',
  'has_thumbnail_custom',
  'video_quality',
  'category',
  'language',
  'has_subtitles',
  'promoted',
  'upload_time',
  'upload_day'])

## 4. Tratamento de Valores Faltantes

### Estratégias aplicadas:
- **Numéricas:** imputação pela **mediana**
- **Categóricas:** imputação pelo **valor mais frequente (moda)**

Motivos:
- Mediana é robusta a outliers  
- Moda garante que categorias novas não são criadas


In [16]:
imputer_num = SimpleImputer(strategy="median")
imputer_cat = SimpleImputer(strategy="most_frequent")

df[numeric_cols] = imputer_num.fit_transform(df[numeric_cols])
df[categorical_cols] = imputer_cat.fit_transform(df[categorical_cols])


## 5. Tratamento de Outliers (Método IQR)

Usamos o método:
- Q1 = 25%
- Q3 = 75%
- IQR = Q3 - Q1  
- Limite inferior = Q1 - 1.5 * IQR  
- Limite superior = Q3 + 1.5 * IQR  

Em vez de remover registros, utilizamos **capping**, substituindo outliers pelos limites permitidos, evitando perda de dados.


In [26]:
df_out = df.copy()

for col in numeric_cols:
    Q1 = df_out[col].quantile(0.25)
    Q3 = df_out[col].quantile(0.75)
    IQR = Q3 - Q1
    
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    
    df_out[col] = np.where(df_out[col] < lower, lower,
                           np.where(df_out[col] > upper, upper, df_out[col]))

df = df_out.copy()


## 6. Encoding de Variáveis Categóricas

Aplicamos **One Hot Encoding (OHE)** para todas as variáveis categóricas nominais.

Usamos:
- `drop='first'` para evitar multicolinearidade  


In [25]:
df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df_encoded.head()


duration_minutes  title_length  description_length  tags_count  \
0               3.0          53.0               589.0        34.0   
1              60.0          95.0               535.0        17.0   
2              47.0          35.0               168.0        20.0   
3               8.0          12.0               260.0         4.0   
4              56.0          94.0               648.0        48.0   

   channel_subscribers  channel_age_months  previous_videos_count  \
0                474.0                59.0                  406.0   
1             899904.0                59.0                   25.0   
2             761732.0                59.0                   59.0   
3             154261.0                84.0                   93.0   
4             843352.0                 6.0                  205.0   

   avg_upload_frequency_days  comments_count  likes_count  shares_count  \
0                        9.0          2804.0       5269.0        2469.5   
1                       14.0          9634.0       9448.0        1679.0   
2                        1.0          6943.0      34028.0         542.0   
3                       10.0          9427.0      42695.0        3096.0   
4                       13.0          2993.0      40761.0         311.0   

   playlist_adds  total_views  video_id_VID00002  video_id_VID00003  \
0          624.0  2103904.125              False              False   
1          894.0   719904.000              False              False   
2          540.0  1420456.000              False              False   
3          188.0   329746.000              False              False   
4          957.0   984985.000              False              False   

   video_id_VID00004  video_id_VID00005  video_id_VID00006  video_id_VID00007  \
0              False              False              False              False   
1              False              False              False              False   
2              False              False              False              False   
3              False              False              False              False   
4              False              False              False              False   

   video_id_VID00008  video_id_VID00009  video_id_VID00010  video_id_VID00011  \
0              False              False              False              False   
1              False              False              False              False   
2              False              False              False              False   
3              False              False              False              False   
4              False              False              False              False   

   video_id_VID00012  video_id_VID00013  video_id_VID00014  video_id_VID00015  \
0              False              False              False              False   
1              False              False              False              False   
2              False              False              False              False   
3              False              False              False              False   
4              False              False              False              False   

   video_id_VID00016  video_id_VID00017  video_id_VID00018  video_id_VID00019  \
0              False              False              False              False   
1              False              False              False              False   
2              False              False              False              False   
3              False              False              False              False   
4              False              False              False              False   

   video_id_VID00020  video_id_VID00021  video_id_VID00022  video_id_VID00023  \
0              False              False              False              False   
1              False              False              False              False   
2              False              False              False              False   
3              False   

## 7. Normalização (StandardScaler)

Aplicamos *StandardScaler*:

- média = 0  
- desvio padrão = 1  

Garante que todas as variáveis numéricas tenham escala semelhante.


In [24]:
import os
from sklearn.preprocessing import StandardScaler
import joblib

# garante que a lista numeric_cols ainda é válida no df_encoded
numeric_cols_present = [c for c in numeric_cols if c in df_encoded.columns]

scaler = StandardScaler()
if len(numeric_cols_present) > 0:
    df_encoded[numeric_cols_present] = scaler.fit_transform(df_encoded[numeric_cols_present])
else:
    print("Atenção: nenhuma coluna numérica encontrada em 'numeric_cols' no dataframe atual.")

# cria a pasta models (um nível acima do notebooks)
os.makedirs("../models", exist_ok=True)

# salva o scaler
joblib.dump(scaler, "../models/scaler.pkl")
print("Scaler salvo em: ../models/scaler.pkl")


Scaler salvo em: ../models/scaler.pkl


## 8. Feature Engineering (Opcional)

Criamos uma nova variável baseada no dataset.
Exemplo criado:
- **engagement_rate = (likes_count + comments_count + shares_count) / total_views**


In [23]:
df_encoded["engagement_rate"] = (
    (df["likes_count"] + df["comments_count"] + df["shares_count"])
    / (df["total_views"] + 1)
)


## 9. Salvando o Dataset Pré-processado

O arquivo final será usado na próxima etapa (modelagem).


In [ ]:
import os

os.makedirs('../data', exist_ok=True)

df_encoded.to_csv("../data/processed/youtube_preprocessed.csv", index=False)
df_encoded.head()
print("="*60)
print("✅ ETAPA 2 CONCLUÍDA!")
print("="*60)
print(f"\n📊 Dataset final:")
print(f"   Registros: {len(df)}")
print(f"   Features: {df.shape[1]}")
print(f"   Missing: {df.isnull().sum().sum()}")
print(f"\n📁 Arquivos gerados:")
print(f"   ✅ data/youtube_preprocessed.csv")
print(f"   ✅ models/scaler.pkl")
print(f"\n🚀 Próximo: Etapa 3 - Modelagem")
print("="*60)



✅ ETAPA 2 CONCLUÍDA!

📊 Dataset final:
   Registros: 2520
   Features: 22
   Missing: 0

📁 Arquivos gerados:
   ✅ data/students_clean.csv
   ✅ models/scaler.pkl

🚀 Próximo: Etapa 3 - Modelagem


# Conclusão da Etapa 2 — Pré-processamento

Nesta etapa realizamos:

- Imputação de valores faltantes  
- Tratamento de outliers com capping pelo IQR  
- One-Hot Encoding para variáveis categóricas  
- Normalização com StandardScaler  
- Criação de feature derivada (engagement_rate)  
- Salvamento do dataset processado  

In [22]:
# %%
# Preview final
print("\n👀 Preview do dataset limpo:")
print(df.head())

print("\nℹ️  Info do dataset:")
print(df.info())



👀 Preview do dataset limpo:
   video_id  duration_minutes  title_length  description_length  tags_count  \
0  VID00197               3.0          53.0               589.0        34.0   
1  VID00987              60.0          95.0               535.0        17.0   
2  VID01091              47.0          35.0               168.0        20.0   
3  VID00388               8.0          12.0               260.0         4.0   
4  VID02222              56.0          94.0               648.0        48.0   

  has_thumbnail_custom video_quality    category   language has_subtitles  \
0                  Sim          720p     Fitness  Português           Não   
1                  Não          720p   Culinária  Português           Não   
2                  Não         1080p      Gaming   Espanhol           Não   
3                  Sim         1080p  Tecnologia  Português           Sim   
4                  Não          720p       Vlogs   Espanhol           Não   

   channel_subscribers  channel_a